In [24]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [26]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Read the CSV and Perform Basic Data Cleaning

In [27]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [28]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [29]:
# Months dictionary
months_num = {
    "Jan-2019": 1,
    "Feb-2019": 2,
    "Mar-2019": 3,
    "Apr-2019": 4,
    "May-2019": 5,
    "Jun-2019": 6,
    "Jul-2019": 7,
    "Aug-2019": 8,
    "Sep-2019": 9,
    "Oct-2019": 10,
    "Nov-2019": 11,
    "Dec-2019": 12,
}

In [30]:
# Months' names encoded using the dictionary values
df['issue_d'] = df['issue_d'].apply(lambda x: months_num[x])
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,3,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,3,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,3,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,3,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,3,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [31]:
# Create our features
X = df.copy()
X = X.drop(["loan_status"], axis=1)
X = pd.get_dummies(X)


# Create our target
y = df["loan_status"]

In [32]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,issue_d,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,1.726172,21.778153,0.217766,0.497697,12.587340,0.126030,...,0.148001,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,0.743862,20.199244,0.718367,0.758122,6.022869,0.336797,...,0.355104,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,1.000000,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,1.000000,13.890000,0.000000,0.000000,8.000000,0.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,2.000000,19.760000,0.000000,0.000000,11.000000,0.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,2.000000,26.660000,0.000000,1.000000,16.000000,0.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,3.000000,999.000000,18.000000,5.000000,72.000000,4.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [33]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [34]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(51612, 93)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [45]:
# Resample the training data with the BalancedRandomForestClassifier
# Create a random forest classifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf = brf.fit(X_train, y_train)
brf

BalancedRandomForestClassifier(random_state=1)

In [50]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = brf.predict(X_test)
brf_score = balanced_accuracy_score(y_test, y_pred)

brf_score

0.7768944567323768

In [47]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   59,    28],
       [ 2129, 14989]])

In [53]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)

print(f'Balanced Random Forest Classifier')
print(f'--------------------------------------------------------------------------')
print(f'Accuracy Score: ') 
display(brf_score)
print(f'--------------------------------------------------------------------------')
print(f'Confusion Matrix:')
display(cm_df)
print(f'--------------------------------------------------------------------------')
print(f'Classification Report:')
print(classification_report_imbalanced(y_test, y_pred))

Balanced Random Forest Classifier
--------------------------------------------------------------------------
Accuracy Score: 


0.7768944567323768

--------------------------------------------------------------------------
Confusion Matrix:


,Predicted High Risk,Predicted Low Risk
Actual High Risk,59,28
Actual Low Risk,2129,14989


--------------------------------------------------------------------------
Classification Report:
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.68      0.88      0.05      0.77      0.58        87
   low_risk       1.00      0.88      0.68      0.93      0.77      0.61     17118

avg / total       0.99      0.87      0.68      0.93      0.77      0.61     17205



In [62]:
# List the features sorted in descending order by feature importance
print(f'Balaned Random Forest Classifier - Features Ranked in Descending Order of Importance')
sorted(zip(brf.feature_importances_,X.columns), reverse=True)

Balaned Random Forest Classifier - Features Ranked in Descending Order of Importance


[(0.06739640738886049, 'total_rec_prncp'),
 (0.06244939886463066, 'total_rec_int'),
 (0.061915356544230964, 'last_pymnt_amnt'),
 (0.05830095090647429, 'total_pymnt_inv'),
 (0.054861602116004285, 'total_pymnt'),
 (0.030509952259938092, 'issue_d'),
 (0.028751564615996813, 'int_rate'),
 (0.01880038917034836, 'out_prncp'),
 (0.01797574649292871, 'annual_inc'),
 (0.017056488322327104, 'total_bc_limit'),
 (0.01690726812806147, 'bc_util'),
 (0.01625885505930925, 'dti'),
 (0.015878685444817842, 'bc_open_to_buy'),
 (0.015479120893631992, 'out_prncp_inv'),
 (0.015439607362141118, 'revol_bal'),
 (0.015368346977184146, 'installment'),
 (0.014691356685661197, 'tot_hi_cred_lim'),
 (0.014473103669181166, 'il_util'),
 (0.014360408690639313, 'mths_since_recent_inq'),
 (0.0143446787858847, 'avg_cur_bal'),
 (0.014323510617082645, 'total_rev_hi_lim'),
 (0.014128224083706876, 'mo_sin_old_il_acct'),
 (0.013961462286895708, 'inq_last_12m'),
 (0.01389945498449954, 'total_bal_ex_mort'),
 (0.013556064983317953,

### Easy Ensemble AdaBoost Classifier

In [55]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
easy_ensemble = EasyEnsembleClassifier(n_estimators = 100, random_state=1)
easy_ensemble = easy_ensemble.fit(X_train, y_train)

In [59]:
# Calculated the balanced accuracy score
y_pred = easy_ensemble.predict(X_test)
ensemble_score = balanced_accuracy_score(y_test, y_pred)
ensemble_score

0.9258362844515352

In [57]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   79,     8],
       [  965, 16153]])

In [60]:
# Print the imbalanced classification report

cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)

print(f'Easy Ensemble AdaBoost Classifier')
print(f'--------------------------------------------------------------------------')
print(f'Accuracy Score: ') 
display(ensemble_score)
print(f'--------------------------------------------------------------------------')
print(f'Confusion Matrix:')
display(cm_df)
print(f'--------------------------------------------------------------------------')
print(f'Classification Report:')
print(classification_report_imbalanced(y_test, y_pred))

Easy Ensemble AdaBoost Classifier
--------------------------------------------------------------------------
Accuracy Score: 


0.9258362844515352

--------------------------------------------------------------------------
Confusion Matrix:


,Predicted High Risk,Predicted Low Risk
Actual High Risk,79,8
Actual Low Risk,965,16153


--------------------------------------------------------------------------
Classification Report:
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.91      0.94      0.14      0.93      0.85        87
   low_risk       1.00      0.94      0.91      0.97      0.93      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.93      0.86     17205



In [63]:
acc_scores = [0.644649, 0.622691, 0.513609, 0.645637, brf_score, ensemble_score]
pre_low = [1.00, 1.00, 1.00, 1.00, 1.00, 1.00]
pre_high = [0.10, 0.10, 0.10, 0.10, 0.03, 0.08]

rec_low = [0.68, 0.65, 0.45, 0.58, 0.88, 0.94]
rec_high = [0.61, 0.60, 0.57, 0.71, 0.68, 0.91]

f1_low = [0.81, 0.79, 0.62, 0.73, 0.93, 0.97]
f1_high = [0.02, 0.02, 0.01, 0.02, 0.05, 0.14]

summary_df = pd.DataFrame(list(zip(acc_scores, pre_low, pre_high, rec_low, rec_high, f1_low, f1_high)),
                         index=["Naive Random Oversampling",
                                "SMOTE Oversampling",
                                "Cluster Centroids Undersampling",
                                "Combination Sampling, SMOTEEN",
                                "Balanced Random Forest Classifier",
                                "Easy Ensemble AdaBoost Classifier"],
                         columns=['Accuracy Score',
                                  'Preciscion (Low Risk)',
                                  'Preciscion (High Risk)',
                                  'Recall (Low Risk)',
                                  'Recall (High Risk)',
                                  'F1 Score (Low Risk)',
                                  'F1 Score (High Risk)'])

summary_df

,Accuracy Score,Preciscion (Low Risk),Preciscion (High Risk),Recall (Low Risk),Recall (High Risk),F1 Score (Low Risk),F1 Score (High Risk)
Naive Random Oversampling,0.644649,1.0,0.10,0.68,0.61,0.81,0.02
SMOTE Oversampling,0.622691,1.0,0.10,0.65,0.60,0.79,0.02
Cluster Centroids Undersampling,0.513609,1.0,0.10,0.45,0.57,0.62,0.01
"Combination Sampling, SMOTEEN",0.645637,1.0,0.10,0.58,0.71,0.73,0.02
Balanced Random Forest Classifier,0.776894,1.0,0.03,0.88,0.68,0.93,0.05
Easy Ensemble AdaBoost Classifier,0.925836,1.0,0.08,0.94,0.91,0.97,0.14


## AdaBoost with More Estimators

In [72]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
easy_ensemble = EasyEnsembleClassifier(n_estimators = 300, random_state=1)
easy_ensemble = easy_ensemble.fit(X_train, y_train)

In [73]:
# Calculated the balanced accuracy score
y_pred = easy_ensemble.predict(X_test)
ensemble_score = balanced_accuracy_score(y_test, y_pred)
ensemble_score

0.926595718964913

In [74]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   79,     8],
       [  939, 16179]])

In [75]:
# Print the imbalanced classification report

cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)

print(f'Easy Ensemble AdaBoost Classifier (300 n_estimators)')
print(f'--------------------------------------------------------------------------')
print(f'Accuracy Score: ') 
display(ensemble_score)
print(f'--------------------------------------------------------------------------')
print(f'Confusion Matrix:')
display(cm_df)
print(f'--------------------------------------------------------------------------')
print(f'Classification Report:')
print(classification_report_imbalanced(y_test, y_pred))

Easy Ensemble AdaBoost Classifier (300 n_estimators)
--------------------------------------------------------------------------
Accuracy Score: 


0.926595718964913

--------------------------------------------------------------------------
Confusion Matrix:


,Predicted High Risk,Predicted Low Risk
Actual High Risk,79,8
Actual Low Risk,939,16179


--------------------------------------------------------------------------
Classification Report:
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.91      0.95      0.14      0.93      0.86        87
   low_risk       1.00      0.95      0.91      0.97      0.93      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.93      0.86     17205

